In [ ]:
import torch

In [3]:
import torch
import matplotlib.pyplot as plt
import numpy as np
import torchvision
from torchvision import datasets
import torchvision.transforms as transforms
import torch.nn as nn
from torchvision import models
import torch.optim as optim
from torchinfo import summary
import mlflow
from torch.utils.data import Subset
torch.set_default_device('cuda')

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [5]:
# remote_server_uri = " http://127.0.0.1:8080"
# mlflow.set_tracking_uri(remote_server_uri)
# mlflow.set_experiment("/mlflow-pytorch-quickstart")

In [6]:
import gc
gc.collect()

293

In [7]:
def tv_split(dataset,split_size=0.25):
    print(len(dataset))
    train_idx, val_idx = torch.utils.data.random_split(dataset,[25787,64])
    datasets = {}
    print(train_idx)
    datasets['train'] = train_idx
    datasets['validation'] = val_idx
    return datasets

In [8]:
transform = transforms.Compose([transforms.Resize((300,300)), transforms.ToTensor(), transforms.RandomHorizontalFlip()])
traindata = datasets.ImageFolder(".\\dataset\\train", transform=transform)
validdata = datasets.ImageFolder(".\\dataset\\valid", transform=transform)

# dataset = tv_split(traindata)
trainloader = torch.utils.data.DataLoader(traindata, batch_size=60, shuffle=True,generator=torch.Generator(device='cuda'))
validationloader = torch.utils.data.DataLoader(validdata, batch_size=60, shuffle=False,generator=torch.Generator(device='cuda'))

loader = {'train':trainloader, 'validation':validationloader}

In [ ]:
def im_convert(tensor):
  img = tensor.clone().detach().numpy()
  img  = img.transpose(1,2,0)
  # img = img*np.array(([0.485, 0.456, 0.406])/np.array(([0.229, 0.224, 0.225])))
  return img

dataiter = iter(trainloader)
images, labels = next(dataiter)
fig = plt.figure(figsize=(25,4))
for i in np.arange(10):
  ax = fig.add_subplot(2,10, i+1)
  plt.imshow(im_convert(images[i]))
  ax.set_title(traindata.classes[labels[i].item()])

In [10]:
model = models.inception_v3(pretrained=False);

In [11]:
in_features = model.fc.in_features
model.fc=nn.Linear(in_features, len(traindata.classes))
model = model.to(device)

In [12]:
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr = 0.01)

In [13]:
running_loss_history = []
val_running_loss_historu = []

running_loss_correct = []
val_running_loss_correct = []
epochs = 20

def train(model, epochs=20):
  for i in range(epochs):
    running_loss = 0.0
    val_running_loss = 0.0

    running_correct = 0.0
    val_running_correct = 0.0

    for inputs, labels in trainloader:
      inputs, labels = inputs.to(device), labels.to(device)
      outputs,_ = model(inputs)
      optimizer.zero_grad()
      loss = loss_fn(outputs, labels)
      loss.backward()
      optimizer.step()
      running_loss+=loss.item()
      _,pred = torch.max(outputs,1)
      running_correct+=torch.sum(pred==labels.data)

    else:
      with torch.no_grad():
        for val_inputs, val_labels in validationloader:
          val_inputs,val_labels = val_inputs.to(device), val_labels.to(device)
          val_outputs,_ = model(val_inputs)
          val_loss = loss_fn(val_outputs, val_labels)
          _,val_pred = torch.max(val_outputs,1)
          val_running_correct+=torch.sum(val_pred==val_labels.data)
          val_running_loss+=val_loss.item()

      val_epoch_accuracy = val_running_correct.float()/len(validationloader.dataset)
      val_epoch_loss = val_running_loss/len(validationloader.dataset)
      val_running_loss_historu.append(val_epoch_loss)
      val_running_loss_correct.append(val_epoch_accuracy)

      epoch_accuracy = running_correct.float()/len(trainloader.dataset)

      epoch_loss = running_loss/len(trainloader.dataset)
      running_loss_history.append(epoch_loss)
      running_loss_correct.append(epoch_accuracy)

      print("Epoch:{}".format(i))
      print("Training loss:{} and Accuracy:{}".format(epoch_loss, epoch_accuracy.item()))
      print("Validation loss: {} and Accuracy:{}".format(val_epoch_loss, val_epoch_accuracy.item()))

In [ ]:
# with mlflow.start_run() as run:
#     params = {
#         "epochs":epochs,
#         "learning_rate":1e-3,
#         "batch_size":60,
#         "loss_function": loss_fn.__class__.__name__,
#         "optimizer": "SGD",
#     }
#     mlflow.log_params(params)
#     with open("model_summary.txt", "w",encoding="utf-8") as f:
#         f.write(str(summary(model)))
#     mlflow.log_artifact("model_summary.txt")
train(model, epochs)
    # mlflow.pytorch.log_model(model, "model")    


Epoch:0
Training loss:0.06634037490958708 and Accuracy:0.29282110929489136
Validation loss: 0.09907288780009767 and Accuracy:0.09396977722644806


In [7]:
model

NameError: name 'model' is not defined